CREO UNA NUOVA COLONNA CON IL GENERE DEGLI ARTISTI - OTTIMIZZATO

In [2]:
import pandas as pd
import wikipedia
import concurrent.futures

# Imposta la lingua italiana
wikipedia.set_lang("it")

# Carica il file Excel e il foglio degli artisti
file_path = "File Unico Sanremo.xlsx"  # Cambia il nome se necessario
df = pd.read_excel(file_path, sheet_name="Artisti unici")

# Cache dei riassunti
summary_cache = {}

def determina_genere(artista):
    if artista in summary_cache:
        return summary_cache[artista]
    
    try:
        # Ottiene il riassunto di Wikipedia
        summary = wikipedia.summary(artista, sentences=2).lower()
        summary_cache[artista] = summary  # Memorizza nella cache
        
        # Logica di identificazione
        if any(x in summary for x in ["cantante", "cantautrice", "attrice", "voce femminile"]):
            return "Femmina"
        elif any(x in summary for x in ["cantautore", "attore", "rapper", "voce maschile"]):
            return "Maschio"
        elif "gruppo" in summary or "band" in summary or "duo" in summary:
            return "Gruppo"
        else:
            return "Da verificare"
    except wikipedia.exceptions.DisambiguationError as e:
        summary_cache[artista] = "Da verificare"
        return "Da verificare"
    except wikipedia.exceptions.PageError:
        summary_cache[artista] = "Da verificare"
        return "Da verificare"

# Funzione per applicare parallelamente
def apply_parallel(df, func):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        return list(executor.map(func, df["Artista"]))

# Applica la funzione in parallelo
df["Genere"] = apply_parallel(df, determina_genere)

# Salva il nuovo file con la colonna aggiunta
df.to_excel("Artisti_unici_con_genere_ottimizzato.xlsx", index=False)

print("✅ Colonna 'Genere' aggiunta e file salvato come 'Artisti_unici_con_genere_ottimizzato.xlsx'")


D:\bocci\anaconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file D:\bocci\anaconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


✅ Colonna 'Genere' aggiunta e file salvato come 'Artisti_unici_con_genere_ottimizzato.xlsx'


In [9]:
import pandas as pd

# Carica il file Excel
file_path = "Artisti_unici_con_genere_ottimizzato.xlsx"
df = pd.read_excel(file_path)

# Mostra le prime righe del DataFrame per verificarlo
print(df.head())  # Mostra le prime 5 righe


                          Artista   Genere
0                     Nilla Pizzi  Femmina
1  Nilla Pizzi e Achille Togliani  Femmina
2                Achille Togliani  Femmina
3   Achille Togliani e Duo Fasano  Femmina
4                      Duo Fasano   Gruppo


In [11]:
print(df)  # Mostra l'intero DataFrame (attenzione se è molto grande!)


                             Artista         Genere
0                        Nilla Pizzi        Femmina
1     Nilla Pizzi e Achille Togliani        Femmina
2                   Achille Togliani        Femmina
3      Achille Togliani e Duo Fasano        Femmina
4                         Duo Fasano         Gruppo
...                              ...            ...
1059                       Gianmaria        Maschio
1060                            Olly        Maschio
1061                            Will  Da verificare
1062                           Shari        Femmina
1063                           Sethu        Femmina

[1064 rows x 2 columns]


In [13]:
print(df['Genere'].value_counts())  # Conta i valori unici nella colonna 'Genere'


Genere
Femmina          511
Maschio          243
Da verificare    199
Gruppo           111
Name: count, dtype: int64
